In [2]:
import iris,requests,metpy.calc,iris.pandas
import iris.plot as iplt
import pandas as pd
import numpy as np
from metpy.units import units
import scipy.interpolate as interp
from trajectory.transforms import i2lla,lla2i

In [45]:
save_loc="data/wind/forcast/gfs"
desired_long=0.1160127
desired_lat=52.2079404


#Who makes an api that when you ask for a grid section too small sends the whole world instead of nothing or rounding it up to a whole one
lat_top=round(desired_lat/.25)*.25
if lat_top>desired_lat:
    lat_bottom=lat_top-.25
else:
    lat_bottom=lat_top+.25
    lat_top,lat_bottom=lat_bottom,lat_top
    
long_left=round(desired_long/.25)*.25
if long_left<desired_long:
    long_right=long_left+.25
else:
    long_right=long_left-.25
    long_left,long_right=long_right,long_left

url="https://nomads.ncep.noaa.gov/cgi-bin/filter_gfs_0p25_1hr.pl?file=gfs.t12z.pgrb2.0p25.f015&lev_0.4_mb=on&lev_1000_mb=on&lev_100_mb=on&lev_10_mb=on&lev_150_mb=on&lev_15_mb=on&lev_180-0_mb_above_ground=on&lev_1_mb=on&lev_200_mb=on&lev_20_mb=on&lev_250_mb=on&lev_255-0_mb_above_ground=on&lev_2_mb=on&lev_300_mb=on&lev_30-0_mb_above_ground=on&lev_30_mb=on&lev_350_mb=on&lev_3_mb=on&lev_400_mb=on&lev_40_mb=on&lev_450_mb=on&lev_500_mb=on&lev_50_mb=on&lev_550_mb=on&lev_5_mb=on&lev_600_mb=on&lev_650_mb=on&lev_700_mb=on&lev_70_mb=on&lev_750_mb=on&lev_7_mb=on&lev_800_mb=on&lev_850_mb=on&lev_900_mb=on&lev_925_mb=on&lev_950_mb=on&lev_975_mb=on&var_HGT=on&var_UGRD=on&var_VGRD=on&subregion=&leftlon={leftlon}&rightlon={rightlon}&toplat={toplat}&bottomlat={bottomlat}&dir=%2Fgfs.20201210%2F12".format(leftlon=long_left,rightlon=long_right,toplat=lat_top,bottomlat=lat_bottom)


In [46]:
r = requests.get(url, stream=True)

with open("%s/1_wind_cam.grb2"%save_loc,'wb') as f:
    for chunk in r.iter_content(chunk_size=1024):
      if chunk: 
         f.write(chunk)

In [47]:
data=iris.load("%s/1_wind_cam.grb2"%save_loc)

In [6]:
lats=list(data[0].coord("latitude").points)
longs=list(data[0].coord("longitude").points)

df=pd.DataFrame(columns=["lat","long","alt","w_x","w_y"])

In [7]:
for long in data[0].coord("longitude").points:
    for lat in lats:
        pres=data[0].extract(iris.Constraint(latitude=lat,longitude=long)).coord("pressure").points
        alts=10*metpy.calc.geopotential_to_height(data[0].extract(iris.Constraint(latitude=lat,longitude=long)).data*units.m**2/units.s**2).magnitude
        for index,alt in enumerate(alts):
            w_x=data[1].extract(iris.Constraint(latitude=lat,longitude=long,pressure=pres[index])).data
            w_y=data[3].extract(iris.Constraint(latitude=lat,longitude=long,pressure=pres[index])).data
            row={"lat":lat,"long":long,"alt":alt,"w_x":w_x,"w_y":w_y}
            df=df.append(row,ignore_index=True)

In [8]:
def get_wind(lat,long,alt):
    lat_query=[None,None]
    lat_query[0]=round(lat/.25)*.25
    if lat_query[0]>lat:
        lat_query[1]=lat_query[0]-.25
    else:
        lat_query[1]=lat_query[0]+.25
        lat_query[0],lat_query[1]=lat_query[1],lat_query[0]
    long_query=[None,None]
    long_query[0]=round(long/.25)*.25
    if long_query[0]<long:
        long_query[1]=long_query[0]+.25
    else:
        long_query[1]=long_query[0]-.25
        long_query[0],long_query[1]=long_query[1],long_query[0]
        
    points = [[None,None],[None,None]]
    for n in [0,1]:
        points[n][0]=df.query('lat == %s'%lat_query[n])
        points[n][1]=df.query('lat == %s'%lat_query[n])
        for m in [0,1]:
            points[n][m]=points[n][m].query('long == %s'%long_query[m])
            
    meaning = [[None,None],[None,None]]
    for n in [0,1]:
        meaning[n][0]="lat_%s"%n
        meaning[n][1]="lat_%s"%n
        for m in [0,1]:
            meaning[n][m]+=", long_%s"%m
            
            
    wind_x = [[None,None],[None,None]]
    wind_y = [[None,None],[None,None]]
    for n in [0,1]:
        for m in [0,1]:
            wind_x[n][m]=interp.interp1d(points[n][m]["alt"],points[n][m]["w_x"])(alt)
            wind_y[n][m]=interp.interp1d(points[n][m]["alt"],points[n][m]["w_y"])(alt)
            
    wind_lats_x = [None,None]
    wind_lats_y = [None,None]
    for n in [0,1]:
        wind_lats_x[n]=interp.interp1d(long_query,wind_x[n])(long)
        wind_lats_y[n]=interp.interp1d(long_query,wind_y[n])(long)
    
    wind_x=interp.interp1d(lat_query,wind_lats_x)(lat)
    wind_y=interp.interp1d(lat_query,wind_lats_y)(lat)
    
    return np.array([wind_x,wind_y,0])
    
    

In [9]:
r_earth = 6378137               #Earth's semimajor axis in meters
ang_vel_earth = 7.292115090e-5  #Earth's angular velocity in rads / s
e_earth = 0.081819191           #Earth ecentricity
f = 1.0/298.257223563 #Flattening of Earth spheroid for WGS84 Model
def lla2i(lat, lon, alt, time):
    # see http://www.mathworks.de/help/toolbox/aeroblks/llatoecefposition.html
    cosLat = np.cos(lat*np.pi/180)
    sinLat = np.sin(lat*np.pi/180)
    FF     = (1.0-f)**2
    C      = 1/np.sqrt(cosLat**2 + FF * sinLat**2)
    S      = C * FF

    x = (r_earth * C + alt)*cosLat * np.cos(lon*np.pi/180+ang_vel_earth*time)
    y = (r_earth * C + alt)*cosLat * np.sin(lon*np.pi/180+ang_vel_earth*time)
    z = (r_earth * S + alt)*sinLat
    return np.array([x, y, z])

def i2lla(pos_i,time):
    #https://uk.mathworks.com/help/aeroblks/ecefpositiontolla.html
    x,y,z=pos_i[0],pos_i[1],pos_i[2]
    longi=np.angle(x+1j*y)
    
    s=np.sqrt(x**2+y**2)
    e=np.sqrt(1-(1-f)**2)
    
    
    beta=np.angle(1j*z+(1-f)*s)
    mu=0
    mu_=np.angle((s-e**2*r_earth*np.cos(beta)**3)+1j*(z+e**2*(1-f)*r_earth*np.sin(beta)**3/(1-e**2)))
    while abs(mu_-mu)>1e-100:#This seems to always converge after one itteration
        mu=mu_
        beta=np.angle(1j*(1-f)*np.sin(mu)+np.cos(mu))
        mu_=np.angle((s-e**2*r_earth*np.cos(beta)**3)+1j*(z+e**2*(1-f)*r_earth*np.sin(beta)**3/(1-e**2)))
    n=r_earth/np.sqrt(1-e**2*np.sin(mu)**2)
    
    h=s*np.cos(mu)+(z+e**2*n*np.sin(mu))*np.sin(mu)-n
    
    lat=mu*180/np.pi
    longi=(longi-ang_vel_earth*time)*180/np.pi
    return lat,longi,h

In [10]:
lat,long,alt=i2lla(np.array([3951980.39029705,6897.51398217,5042787.58273108]),0)
get_wind(lat,long,alt)

array([107.04732326,   1.27590173,   0.        ])

In [48]:
data

[<iris 'Cube' of geopotential_height / (m) (pressure: 34; latitude: 2; longitude: 2)>,
<iris 'Cube' of x_wind / (m s-1) (pressure: 31; latitude: 2; longitude: 2)>,
<iris 'Cube' of x_wind / (m s-1) (latitude: 2; longitude: 2)>,
<iris 'Cube' of y_wind / (m s-1) (pressure: 31; latitude: 2; longitude: 2)>,
<iris 'Cube' of y_wind / (m s-1) (latitude: 2; longitude: 2)>]

In [19]:
df.iloc[(df["alt"]-0).abs().argsort()[:1]]["w_x"].astype(float).to_numpy()[0]

-3.4626535034179686

In [15]:
df["alt"]

0      45870.887533
1      40590.797230
2      37695.672081
3      34299.172602
4      32192.929428
           ...     
119      887.482344
120      661.975988
121      441.139777
122      225.080970
123       14.732959
Name: alt, Length: 124, dtype: float64

In [73]:
data2=iris.load("/Users/jago/Documents/Projects/6DOF-Trajectory-Simulation/data/wind/gfs/52.0_0.0_00_001.grb2")

In [74]:
data2[1][0][0][0]

X Wind (m s-1),(scalar cube)
Scalar coordinates,
forecast_period,1 hours
forecast_reference_time,2020-12-13 00:00:00
latitude,52.0 degrees
longitude,0.0 degrees
pressure,100.0 Pa
time,2020-12-13 01:00:00
Attributes,
GRIB_PARAM,GRIB2:d000c002n002


In [53]:
[4.00e+01, 1.00e+02, 2.00e+02, 3.00e+02, 5.00e+02, 7.00e+02,
       1.00e+03, 1.50e+03, 2.00e+03, 3.00e+03, 4.00e+03, 5.00e+03,
       7.00e+03, 1.00e+04, 1.50e+04, 2.00e+04, 2.50e+04, 3.00e+04,
       3.50e+04, 4.00e+04, 4.50e+04, 5.00e+04, 5.50e+04, 6.00e+04,
       6.50e+04, 7.00e+04, 7.50e+04, 8.00e+04, 8.50e+04, 9.00e+04,
       9.25e+04, 9.50e+04, 9.75e+04, 1.00e+05]

[<iris 'Cube' of geopotential_height / (m) (pressure: 34; latitude: 2; longitude: 2)>,
<iris 'Cube' of x_wind / (m s-1) (pressure: 31; latitude: 2; longitude: 2)>,
<iris 'Cube' of x_wind / (m s-1) (latitude: 2; longitude: 2)>,
<iris 'Cube' of y_wind / (m s-1) (pressure: 31; latitude: 2; longitude: 2)>,
<iris 'Cube' of y_wind / (m s-1) (latitude: 2; longitude: 2)>]